In [ ]:
import tensorflow as tf
import easydict
import numpy as np

In [ ]:
def load_pb(path_to_pb):
    """Load the .pb file and return graph

    :param path_to_pb: The path of the .pb file
    :type path_to_pb: str
    :return: graph
    :rtype: GraphDef
    """
    with tf.compat.v1.gfile.GFile(path_to_pb, "rb") as f:
        graph_def = tf.compat.v1.GraphDef()
        graph_def.ParseFromString(f.read())
    with tf.compat.v1.Graph().as_default() as graph:
        tf.compat.v1.import_graph_def(graph_def, name='')
        return graph, graph_def

def analyze_inputs_outputs(graph):
    """Analyzing the graph and return the input node and output node

    :param graph: The computation graph
    :type graph: GraphDef
    :return: [Input Node List, Output Node List]
    :rtype: List
    """
    ops = graph.get_operations()
    outputs_set = set(ops)
    inputs = []
    for op in ops:
        if len(op.inputs) == 0 and op.type != 'Const':
            inputs.append(op)
        else:
            for input_tensor in op.inputs:
                if input_tensor.op in outputs_set:
                    outputs_set.remove(input_tensor.op)
    outputs = list(outputs_set)
    return (inputs, outputs)

In [ ]:
path = "/opt/Anonymous/workspace/mob-dl-rev/data/raw/tensorflow_model/2983d8d0b3d412cf548840100fa07673/tfImg.pb"
graph = load_pb(path)
input_nodes, output_nodes = analyze_inputs_outputs(graph)
print("Input Number: {}".format(len(input_nodes)))
print("Output Number: {}".format(len(output_nodes)))

In [ ]:
for in_node in input_nodes:
    print("Input Node Name: " + in_node.name)
for out_node in output_nodes:
    print("Out Node Name: " + out_node.name)


In [ ]:
input_tensor = graph.get_tensor_by_name('input:0')
output_tensor = graph.get_tensor_by_name('output:0')

In [ ]:
sess = tf.compat.v1.Session()
tf.compat.v1.import_graph_def(graph)
input_image = tf.constant(np.random.random(size=(1,224,224,3)))
sess.run(output_tensor, feed_dict={input_tensor: input_image})